In [11]:
import numpy as np
import pandas as pd
import os

In [14]:
file_path_label = '../../diabetic/Diabetic-Retinopathy-Detection/archive/trainLabels_cropped.csv'

In [15]:
home_data_label = pd.read_csv(file_path_label)
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex3 import *

print("Setup Complete")

Setup Complete
